In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, RepeatVector, TimeDistributed, Concatenate, Bidirectional, Activation, Multiply
import tensorflow as tf

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Parameters
num_samples = 1000  # Total number of samples
sequence_length = 50  # Length of each sequence

# Generate synthetic data
packet_sizes = np.random.normal(loc=500, scale=100, size=num_samples)
connection_durations = np.random.normal(loc=2, scale=0.5, size=num_samples)

# Generate timestamps (1-second intervals for simplicity)
timestamps = pd.date_range(start='2024-11-05', periods=num_samples, freq='S')

# Calculate inter-arrival times (difference between consecutive timestamps)
inter_arrival_times = [0] + [(timestamps[i] - timestamps[i-1]).total_seconds() for i in range(1, num_samples)]

# Calculate packet size to connection duration ratio (add small constant to avoid division by zero)
size_duration_ratios = packet_sizes / (connection_durations + 1e-5)

# Generate SYN packet flags randomly (probability of 10% to be a SYN packet)
syn_flags = np.random.choice([0, 1], size=num_samples, p=[0.9, 0.1])

# Calculate SYN packet frequency within each sequence
syn_frequencies = []
for i in range(num_samples):
    # Calculate the rolling frequency of SYN packets over the last `sequence_length` packets
    start_idx = max(0, i - sequence_length + 1)
    syn_frequency = np.sum(syn_flags[start_idx:i+1]) / sequence_length
    syn_frequencies.append(syn_frequency)

# Create DataFrame with all features
df = pd.DataFrame({
    'packet_size': packet_sizes,
    'connection_duration': connection_durations,
    'inter_arrival_time': inter_arrival_times,
    'size_duration_ratio': size_duration_ratios,
    'syn_flag': syn_flags,  # Indicator of whether each packet is a SYN packet
    'syn_frequency': syn_frequencies  # Rolling frequency of SYN packets
})

In [ ]:

# Assuming your dataset includes the following columns
# 'packet_size', 'connection_duration', 'packet_inter_arrival_time', 'SYN_packets', 'packet_size_connection_ratio', 'SYN_packet_ratio'

# Load the dataset (assuming you have the dataset as a CSV or DataFrame)
# For example: df = pd.read_csv("synthetic_network_traffic.csv")
# For the sake of this example, I'll create a sample dataframe

# Example DataFrame with features
num_samples = 1000  # Assuming 1000 samples
sequence_length = 50  # Length of the sequence

# Create synthetic data (example)
df = pd.DataFrame({
    'packet_size': np.random.normal(loc=500, scale=100, size=num_samples),
    'connection_duration': np.random.normal(loc=2, scale=0.5, size=num_samples),
    'packet_inter_arrival_time': np.random.normal(loc=0.02, scale=0.005, size=num_samples),
    'SYN_packets': np.random.randint(0, 10, size=num_samples),
    'packet_size_connection_ratio': np.random.normal(loc=250, scale=50, size=num_samples),
    'SYN_packet_ratio': np.random.uniform(0, 1, size=num_samples)
})

# Scale the data (Min-Max Scaling for neural network input)
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df)  # Normalize the features

# Prepare X_train_data (sequence of 6 features per time step)
X_train_data = []
for i in range(num_samples - sequence_length):
    X_train_data.append(scaled_features[i:i + sequence_length])

X_train_data = np.array(X_train_data)  # Shape should be (num_samples-sequence_length, sequence_length, 6)

# Prepare y_train_data (packet_size and connection_duration as targets)
y_train_data = []
for i in range(num_samples - sequence_length):
    # We take only the 'packet_size' and 'connection_duration' columns for the target
    y_train_data.append(scaled_features[i:i + sequence_length, [0, 1]])

y_train_data = np.array(y_train_data)  # Shape should be (num_samples-sequence_length, sequence_length, 2)

# Ensure the shape of the data
print("Shape of X_train_data:", X_train_data.shape)  # Expected: (950, 50, 6)
print("Shape of y_train_data:", y_train_data.shape)  # Expected: (950, 50, 2)


Shape of X_train_data: (950, 50, 6)
Shape of y_train_data: (950, 50, 2)


In [6]:
# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)  # Scale all features

# Create sequences
X = []
for i in range(num_samples - sequence_length + 1):  # Adjust range to include the last sequence
    X.append(data_scaled[i:i + sequence_length])

X_train = np.array(X)  # Convert list to NumPy array

# Check the shape to confirm it's correct
print("Shape of training data:", X_train.shape)  # Expected shape: (951, 50, 6)

Shape of training data: (951, 50, 6)


In [25]:
# Attention Layer Definition
def attention_layer(inputs):
    """
    Applies an attention mechanism on the input sequence.
    """
    attention = Dense(1, activation="tanh")(inputs)
    attention = tf.keras.layers.Flatten()(attention)
    attention = Activation("softmax")(attention)
    attention = tf.keras.layers.RepeatVector(inputs.shape[-1])(attention)
    attention = tf.keras.layers.Permute([2, 1])(attention)

    output_attention = Multiply()([inputs, attention])
    return output_attention

In [26]:
input_layer = Input(shape=(sequence_length, 6))

In [27]:
# LSTM branch
lstm_branch = LSTM(64, activation='relu', return_sequences=True)(input_layer)
lstm_branch = LSTM(32, activation='relu', return_sequences=False)(lstm_branch)
lstm_branch = RepeatVector(sequence_length)(lstm_branch)
lstm_branch = LSTM(32, activation='relu', return_sequences=True)(lstm_branch)
lstm_branch = LSTM(64, activation='relu', return_sequences=True)(lstm_branch)

In [28]:
# Bidirectional LSTM branch
bidirectional_branch = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_layer)
bidirectional_branch = Bidirectional(LSTM(32, activation='relu', return_sequences=False))(bidirectional_branch)
bidirectional_branch = RepeatVector(sequence_length)(bidirectional_branch)
bidirectional_branch = Bidirectional(LSTM(32, activation='relu', return_sequences=True))(bidirectional_branch)
bidirectional_branch = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(bidirectional_branch)

In [29]:
# GRU branch
gru_branch = GRU(64, activation='relu', return_sequences=True)(input_layer)
gru_branch = GRU(32, activation='relu', return_sequences=False)(gru_branch)
gru_branch = RepeatVector(sequence_length)(gru_branch)
gru_branch = GRU(32, activation='relu', return_sequences=True)(gru_branch)
gru_branch = GRU(64, activation='relu', return_sequences=True)(gru_branch)

In [30]:
# Concatenate all branches
combined = Concatenate()([lstm_branch, bidirectional_branch, gru_branch])

# Apply attention mechanism
attention_output = attention_layer(combined)

# Final TimeDistributed layer
output_layer = TimeDistributed(Dense(2))(attention_output)


In [31]:
# Build the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 50, 6)]              0         []                            
                                                                                                  
 lstm_16 (LSTM)              (None, 50, 64)               18176     ['input_3[0][0]']             
                                                                                                  
 bidirectional_8 (Bidirecti  (None, 50, 128)              36352     ['input_3[0][0]']             
 onal)                                                                                            
                                                                                                  
 gru_8 (GRU)                 (None, 50, 64)               13824     ['input_3[0][0]']       

 ional)                                                                                           
                                                                                                  
 gru_10 (GRU)                (None, 50, 32)               6336      ['repeat_vector_10[0][0]']    
                                                                                                  
 lstm_19 (LSTM)              (None, 50, 64)               24832     ['lstm_18[0][0]']             
                                                                                                  
 bidirectional_11 (Bidirect  (None, 50, 128)              66048     ['bidirectional_10[0][0]']    
 ional)                                                                                           
                                                                                                  
 gru_11 (GRU)                (None, 50, 64)               18816     ['gru_10[0][0]']              
          

In [22]:
# Split into training and validation sets manually if shuffle is causing issues
train_size = int(0.9 * len(X_train))
X_train_data, X_val_data = X_train[:train_size], X_train[train_size:]

In [ ]:
# Fit the model
history = model.fit(
    X_train_data, y_train_data,  # X_train_data as both input and target
    epochs=50,
    batch_size=32,
    validation_split=0.1,  # Use part of the data for validation
    shuffle=True  # Shuffle for randomized training batches
)

Epoch 1/50
27/27 [==============================] - 29s 253ms/step - loss: 0.8655 - val_loss: 0.1707
Epoch 2/50
27/27 [==============================] - 5s 174ms/step - loss: 0.1688 - val_loss: 0.1405
Epoch 3/50
27/27 [==============================] - 5s 200ms/step - loss: 0.0888 - val_loss: 0.0251
Epoch 4/50
27/27 [==============================] - 5s 189ms/step - loss: 0.0244 - val_loss: 0.0217
Epoch 5/50
27/27 [==============================] - 5s 195ms/step - loss: 0.0228 - val_loss: 0.0208
Epoch 6/50
27/27 [==============================] - 6s 210ms/step - loss: 0.0223 - val_loss: 0.0206
Epoch 7/50
27/27 [==============================] - 6s 230ms/step - loss: 0.0221 - val_loss: 0.0204
Epoch 8/50
27/27 [==============================] - 5s 184ms/step - loss: 0.0219 - val_loss: 0.0204
Epoch 9/50
27/27 [==============================] - 5s 198ms/step - loss: 0.0218 - val_loss: 0.0203
Epoch 10/50
27/27 [==============================] - 6s 228ms/step - loss: 0.0218 - val_loss: 0.020

In [33]:
model.save('model3.h5')

C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
